# Example for calculating the fn-coefficients

In the following, a simple example on how to manually evaluate the fn-coefficients is given. 
The ground is hereby defined as a Lambertian-surface and the covering layer is assumed to consist of Rayleigh-particles. Thus, we have: ($R_0$ hereby denotes the diffuse albedo of the surface)

Within the following calculation, the symbols and functions are defined as presented in the Theory-chapter of the documentation.

### Definition of the BRDF of the ground surface:

- $BRDF(\theta, \phi, \theta_{ex},\phi_{ex}) = \frac{R_0}{\pi}$

### Definition of the scattering phase-function of the covering layer:

- $p(\theta, \phi, \theta_{ex},\phi_{ex}) = \frac{3}{16\pi} (1+\cos(\Theta)^2) \quad$ with $\mbox{}\quad$ $\cos(\Theta) = \cos(\theta)\cos(\theta_{ex}) + \sin(\theta)\sin(\theta_{ex})\cos(\phi - \phi_{ex})$

### Manual Calculation of the fn-coefficients:

$INT := \int_0^{2\pi} p(\theta_0, \phi_0, \theta,\phi) * BRDF(\pi-\theta, \phi, \theta_{ex},\phi_{ex}) d\phi$

$\phantom{INT} = \frac{3 R_0}{16 \pi^2} \int_{0}^{2\pi}  (1+[\cos(\theta_0)\cos(\theta) + \sin(\theta_0)\sin(\theta)\cos(\phi_0 - \phi)]^2) d\phi$

$\phantom{INT} = \frac{3 R_0}{16 \pi^2} \int_0^{2\pi} (1+ \mu_0^2 \mu^2 + 2 \mu_0 \mu \sin(\theta_0) \sin(\theta) \cos(\phi_0 - \phi) + (1-\mu_0)^2(1-\mu)^2 \cos(\phi_0 - \phi)^2 d\phi$


where the shorthand-notation $\mu_x = \cos(\theta_x)$ has been introduced.
The above integral can now easily be solved by noticing:

$\int_0^{2\pi} \cos(\phi_0 - \phi)^n d\phi = \left\lbrace \begin{matrix} 2 \pi & \textrm{for } n=0 \\ 0 & \textrm{for } n=1 \\ \pi  & \textrm{for } n=2 \end{matrix} \right.$
    
Inserting the above solution and using some algebraic manipulations we therefore find:

$INT = \frac{3 R_0}{16\pi} \Big[ (3-\mu_0^2) + (3 \mu_0 -1) \mu^2 \Big] := R_0 ~\sum_{n=0}^2 f_n \, \mu^n$

$\Rightarrow \quad f_0 = \frac{3(3-\mu_0^2)}{16\pi} \qquad f_1 = 0 \qquad f_2 = \frac{3(3 \mu_0 -1)}{16\pi} \qquad f_n = 0 \, \forall \, n>2$

## Evaluation of the fn-coefficients using the RT1-module

In [1]:
# imports
from rt1_model import RT1, volume, surface
import sympy as sp
# enable printing sympy equations via latex-equation-rendering
sp.init_printing(use_latex='mathjax')

In [2]:
# definition of volume and surface
V = volume.Rayleigh()
SRF = surface.Isotropic()
R = RT1(V=V, SRF=SRF)  
R._fn

[0.0 + 0.1875*sin(theta_0)**2/pi + 0.375*pi**(-1),
 0.0 + 0.1875*sin(theta_0)**2/pi - (0.0 + 0.1875*sin(theta_0)**2/pi + 0.375*pi**(-1)) + 0.375*pi**(-1),
 0.0 + 0.375*cos(theta_0)**2/pi - (0.0 + 0.1875*sin(theta_0)**2/pi + 0.375*pi**(-1)) + 0.375*pi**(-1)]

In [3]:
# apply basic trigonometric simplifications to see that the coefficients are really equal
fn = [sp.trigsimp(i) for i in R._fn]
# replace appearing sin^2 with 1-cos^2
fn = [i.xreplace({sp.sin('theta_0')**2 : 1.-sp.cos('theta_0')**2}) for i in fn] 
# convert floats to rationals and factor out
fn = [i.nsimplify().factor() for i in fn]
fn = [i.xreplace({sp.cos('theta_0') : sp.Symbol("mu_0")}) for i in fn] 
fn = [i.simplify() for i in fn]
fn

⎡  ⎛      2⎞       ⎛    2    ⎞⎤
⎢3⋅⎝3 - μ₀ ⎠     3⋅⎝3⋅μ₀  - 1⎠⎥
⎢───────────, 0, ─────────────⎥
⎣    16⋅π             16⋅π    ⎦